In [4]:
options(digits = 2)
library(foreach)
library(doMC)
registerDoMC(cores = 4)

Synthetic Facebook data set from https://www.udacity.com/wiki/ud651#!#data-sets.
We are working with a subset of this to make the simulations fast.

In [2]:
ps <- read.table("pseudo_facebook_small.tsv", header = TRUE)

In [20]:
dim(ps)
summary(ps)

[1] 10000    15

     userid             age         dob_day        dob_year      dob_month       gender         tenure    
 Min.   :1000406   Min.   : 13   Min.   : 1.0   Min.   :1900   Min.   : 1.0   female:4028   Min.   :   0  
 1st Qu.:1299415   1st Qu.: 21   1st Qu.: 6.0   1st Qu.:1963   1st Qu.: 3.0   male  :5952   1st Qu.: 229  
 Median :1600884   Median : 29   Median :14.0   Median :1984   Median : 6.0   NA's  :  20   Median : 411  
 Mean   :1601339   Mean   : 37   Mean   :14.5   Mean   :1976   Mean   : 6.3                 Mean   : 539  
 3rd Qu.:1900566   3rd Qu.: 50   3rd Qu.:22.0   3rd Qu.:1992   3rd Qu.: 9.0                 3rd Qu.: 672  
 Max.   :2193485   Max.   :113   Max.   :31.0   Max.   :2000   Max.   :12.0                 Max.   :2822  
  friend_count  friendships_initiated     likes       likes_received   mobile_likes   mobile_likes_received
 Min.   :   0   Min.   :   0          Min.   :    0   Min.   :    0   Min.   :    0   Min.   :    0        
 1st Qu.:  30   1st Qu.:  16       

Simulation that computes simple difference in means and regression adjustment estimators.

In [9]:
do.sim <- function(x, y, nt = n / 2) {
    n <- length(x)
    
    z.c <- rep(0, n)
    z.c[sample.int(n, nt)] <- 1
    
    tau.sd <- unname(coef(lm(y ~ z.c))[2])
    tau.adj <- unname(coef(lm(y ~ z.c + x))[2])
    
    x0 <- x - mean(x)
    tau.adj.int <- unname(coef(lm(y ~ z.c * x0))[2])
    
    c(th.sd = tau.sd, th.adj = tau.adj, th.adj.int = tau.adj.int)
}

Do a single simulation:

In [21]:
with(ps, do.sim(log1p(tenure), log1p(likes), nt = 2000))

th.sd     th.adj th.adj.int 
     0.040      0.049      0.049

Do many simulations:

In [22]:
sr <- foreach(i = 1:100, .combine = rbind) %dopar%
  with(ps, do.sim(log1p(tenure), log1p(likes), nt = 2000))

In [23]:
summary(sr)

     th.sd            th.adj         th.adj.int    
 Min.   :-0.127   Min.   :-0.129   Min.   :-0.129  
 1st Qu.:-0.035   1st Qu.:-0.037   1st Qu.:-0.038  
 Median : 0.000   Median : 0.003   Median : 0.002  
 Mean   : 0.002   Mean   : 0.002   Mean   : 0.002  
 3rd Qu.: 0.041   3rd Qu.: 0.038   3rd Qu.: 0.038  
 Max.   : 0.159   Max.   : 0.148   Max.   : 0.149  

In [24]:
apply(sr, 2, sd)

th.sd     th.adj th.adj.int 
     0.060      0.057      0.057

In [25]:
apply(sr, 2, mean)

th.sd     th.adj th.adj.int 
    0.0015     0.0020     0.0018

In [26]:
mses <- apply(sr, 2, function(x) sum(x^2))
mses

th.sd     th.adj th.adj.int 
      0.35       0.32       0.32

In [28]:
1 - mses / mses[1]

th.sd     th.adj th.adj.int 
     0.000      0.085      0.083